In [1]:
import numpy as np
import pandas as pd
import os
import re

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Phrases
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import FastText


from keras.models import Sequential
import gensim
from gensim.models import FastText
from keras.layers import Embedding,Dense,LSTM, Dropout, Flatten, Convolution1D, MaxPooling1D


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
%matplotlib inline

Using TensorFlow backend.
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [ ]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('wiki.en/wiki.en.bin')

### preprocessing data

In [2]:
#Preprocessing - removing unwanted characters, tokenization, stop-word removal
def clean_data(txts):
    x = re.sub("[^a-zA-Z]", " ",txts) 
    x = x.lower().split()                             
    stops = set(stopwords.words("english"))                  
    words = [w for w in x if not w in stops] 
    
    return( " ".join(words))

In [3]:
def clean_norm(txts):
    x = re.sub('(OntoBiotope |)','',txts)
    x = re.sub('(:|_)',' ',x)
    x = re.sub('(Taxonomy)','taxon',x)
    
    return x

### Train_norm+Dev_norm appended (same as Train_norm+ner, Dev_norm+ner appended)

In [4]:
data = pd.read_csv('data.csv')

data['norm2'].fillna(value='', inplace=True)
data['norm3'].fillna(value='', inplace=True)
data['norm4'].fillna(value='', inplace=True)
data['norm5'].fillna(value='', inplace=True)

data = data.drop(data[data.entity == 'Paragraph'].index)
data = data.drop(data[data.entity == 'Title'].index)
print(data.shape)
data.head()

(3268, 7)


,words,entity,norm1,norm2,norm3,norm4,norm5
0,selective broths based on hypertonic strontium...,Habitat,OntoBiotope OBT:000360,,,,
1,selective broths based on the bi-selenite ion,Habitat,OntoBiotope OBT:000360,,,,
2,Salmonellae,Microorganism,NCBI_Taxonomy 590,,,,
3,egg products,Habitat,OntoBiotope OBT:001086,,,,
4,egg,Habitat,OntoBiotope OBT:001847,,,,


### Test_norm

In [ ]:
def get_test_data(path):
    z=[]
    df_dat = pd.DataFrame(columns = ['text','words','entity'])
    org_dat = os.listdir(path)
    for i in range(len(org_dat)):
        org1 = org_dat[i]
        if org1.endswith('.a1'):
            org_path1 = os.path.join(path,org1)
            text1 = pd.read_table(org_path1,header = None)
            for j in range(0,text1.shape[0]):
                #print(org1,org2)
                #print(text1.iloc[j][2],re.sub(r'[0-9; ]','',text1.iloc[j][1]))
                z.append(org1)
                df_dat = df_dat.append({'text':text1.iloc[j][0] ,'words': text1.iloc[j][2], 'entity': re.sub(r'[0-9; ]','',text1.iloc[j][1]) }, ignore_index = True)
                #print(df_dat)
        
    return df_dat,z  

In [ ]:
path = 'BioNLP-OST-2019_BB-norm_test'
df_test,z = get_test_data(path)

print(df_test.shape)

df_test = df_test.drop(df_test[df_test.entity == 'Paragraph'].index)
df_test = df_test.drop(df_test[df_test.entity == 'Title'].index)
print(df_test.shape)
df_test.head()

In [5]:
# test = pd.read_csv('test.csv')
# print(test.shape)

# test = test.drop(test[test.entity == 'Paragraph'].index)
# test = test.drop(test[test.entity == 'Title'].index)
# print(test.shape)
# test.head()

(1977, 2)
(1818, 2)


,words,entity
2,Escherichia coli,Microorganism
3,bacterial pathogenicity,Phenotype
4,infected germfree piglets,Habitat
5,Escherichia coli,Microorganism
6,intestinal,Habitat


In [6]:
data['words'] = data['words'].apply(lambda x: clean_data(x))
test = df_test
test['words'] = test['words'].apply(lambda x: clean_data(x))

In [8]:
data_words = data['words'].tolist()
data_entity = data['entity'].tolist()

test_words = test['words'].tolist()
test_entity = test['entity'].tolist()

In [9]:
X1_train= np.zeros((len(data_words),300)) 
X2_train= np.zeros((len(data_entity),300)) 

for i in range(len(data_words)):
    words = word_tokenize(data_words[i])
    emb = [model.wv[w] for w in words] 
    X1_train[i] = np.mean(emb, axis=0) 
    
for w in data_entity:
    X2_train[i] = model.wv[w]
    
X_train= np.concatenate((X1_train,X2_train),axis=1)
print(X_train.shape)

X1_test= np.zeros((len(test_words),300)) 
X2_test= np.zeros((len(test_entity),300)) 

for i in range(len(test_words)):
    words = word_tokenize(test_words[i])
    emb = [model.wv[w] for w in words] 
    X1_test[i] = np.mean(emb, axis=0) 
    
for w in test_entity:
    X2_test[i] = model.wv[w]  
    
    
X_test = np.concatenate((X1_test,X2_test),axis=1)
print(X_test.shape)


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(3268, 600)
(1818, 600)


In [10]:
df_train= pd.DataFrame(X_train)
df_test= pd.DataFrame(X_test)

In [11]:
df_train.fillna(value=0,inplace=True)
df_test.fillna(value=0,inplace=True)

In [12]:
X_test = np.array(df_test)
X_train = np.array(df_train)
print(X_test.shape)
print(X_train.shape)

(1818, 600)
(3268, 600)


In [13]:
data_norm1 = data['norm1'].tolist()

In [14]:
le = LabelEncoder()
Y = le.fit_transform(data_norm1)

Y.shape

(3268,)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [16]:
model_class = RandomForestClassifier(n_estimators=30,criterion='entropy',class_weight='balanced')
model_class.fit(X_train,Y)
pre = model_class.predict(X_test)
# print(accuracy_score(y_test,pre))
# print(confusion_matrix(y_test,pre))

In [17]:
norm = le.inverse_transform(pre)

In [18]:
sub = pd.DataFrame({'words':test['words'],'entity':test['entity'],'norm':norm})

In [62]:
sub.head()

,0
0,N1\tNCBI_Taxonomy Annotation:T3 Referent:562
1,N1\tOntoBiotope Annotation:T4 Referent:OBT:000375
2,N2\tOntoBiotope Annotation:T5 Referent:OBT:003206
3,N4\tNCBI_Taxonomy Annotation:T6 Referent:562
4,N4\tOntoBiotope Annotation:T7 Referent:OBT:002810


In [24]:
# def get_test_data(path):
#     z=[]
#     df_dat = pd.DataFrame(columns = ['text','words','entity'])
#     org_dat = os.listdir(path)
#     for i in range(len(org_dat)):
#         org1 = org_dat[i]
#         if org1.endswith('.a1'):
#             org_path1 = os.path.join(path,org1)
#             text1 = pd.read_table(org_path1,header = None)
#             for j in range(0,text1.shape[0]):
#                 #print(org1,org2)
#                 #print(text1.iloc[j][2],re.sub(r'[0-9; ]','',text1.iloc[j][1]))
#                 z.append(org1)
#                 df_dat = df_dat.append({'text':text1.iloc[j][0] ,'words': text1.iloc[j][2], 'entity': re.sub(r'[0-9; ]','',text1.iloc[j][1]) }, ignore_index = True)
#                 #print(df_dat)
        
#     return df_dat,z  

In [27]:
# path = 'BioNLP-OST-2019_BB-norm_test'
# df_test,z = get_test_data(path)

# print(df_test.shape)

# df_test = df_test.drop(df_test[df_test.entity == 'Paragraph'].index)
# df_test = df_test.drop(df_test[df_test.entity == 'Title'].index)
# print(df_test.shape)
# df_test.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  if __name__ == '__main__':


(1977, 3)
(1818, 3)


,text,words,entity
2,T3,Escherichia coli,Microorganism
3,T4,bacterial pathogenicity,Phenotype
4,T5,infected germfree piglets,Habitat
5,T6,Escherichia coli,Microorganism
6,T7,intestinal,Habitat


In [38]:
text = df_test['text'].tolist()

In [59]:
annotation=[]
for i in range(0,norm.shape[0]):
    if 'NCBI' in norm[i]:
        annotation.append(f'N{i+1}\tNCBI_Taxonomy Annotation:{text[i]} Referent:{os.path.splitext(norm[i])[0][14:]}')
        
    elif 'OntoBiotope' in norm[i]:
        annotation.append(f'N{i}\tOntoBiotope Annotation:{text[i]} Referent:{os.path.splitext(norm[i])[0][12:]}')
    

In [60]:
sub = pd.DataFrame(annotation)

In [ ]:
sub.head()

In [ ]:
pd.read_csv('BioNLP-OST-2019_BB-norm_train/BB-norm-448557.a1',header=None)

In [46]:
x_train,x_test,y_train,y_test = train_test_split(X_train,Y,test_size=0.10,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2941, 600), (327, 600), (2941,), (327,))

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [48]:
model_class = RandomForestClassifier(n_estimators=30,criterion='entropy',class_weight='balanced')
model_class.fit(x_train,y_train)
pre = model_class.predict(x_test)
print(accuracy_score(y_test,pre))
print(confusion_matrix(y_test,pre))

0.7125382262996942
[[4 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 2]]


In [49]:
norm = le.inverse_transform(pre)

In [ ]:
sub = pd.DataFrame({'words':test['words'],'entity':test['entity'],'norm':norm})

## cross validation

In [1]:
from sklearn.model_selection import cross_val_predict